In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__, "..")); Pkg.instantiate();
using RobotZoo: YakPlane
import RobotDynamics
using RobotDynamics: state_dim, control_dim
using ForwardDiff
using StaticArrays
using LinearAlgebra
using Rotations
using Printf
using Test
using TrajOptPlots
using JLD2
const RD = RobotDynamics
include("utils.jl")
include("airplane.jl")
const isautograder = @isdefined autograder;
const resfile = joinpath(@__DIR__,"q1.jld2")

  Activating project at `~/Documents/Robotics PhD/Fourth Semester/15-780 Grad AI/Final Project`


"/Users/pfisch/Documents/Robotics PhD/Fourth Semester/15-780 Grad AI/Final Project/Grad_AI_Final_Project/q1.jld2"

In [2]:
# Dynamics API
let
    model = YakPlane(UnitQuaternion)
    x,_ = rand(model)  # generate a random state and control input (with the orientation sampled correctly)
    u = Vector{Float64}(rand(0:255, 4))
    t  = 0.0  # time
    dt = 0.1  # time step
    
    # continuous dynamics
    dynamics(model, x, u)
    
    # discrete dynamics
    xnext = discrete_dynamics(model, x, u, t, dt);
    A,B = discrete_jacobian(model::YakPlane, x, u, t, dt)
    
end;  # wrap in let/end to avoid polluting the global state

In [3]:

"""
    get_trim(model::YakPlane, x_trim, u_guess)

Calculate the trim controls for the YakPlane model, given the position and velocity specified by `x_trim`.
Find a vector of 4 controls close to those in `u_guess` that minimize the accelerations on the plane.
"""
function get_trim(model::YakPlane, x_trim, u_guess;
        verbose = false,
        tol = 1e-4,
        iters = 100,
    )

    λ = zero(x_trim[8:13])
    
    x_num = state_dim(model) 
    u_num = control_dim(model)
    utrim = copy(u_guess)
    acc_num = 6 #number of accelerations in the state
    ρ = 1e-5 #regularizing term
    

    for k = 1:iters
        
        B = ForwardDiff.jacobian(dutrim -> dynamics(model, x_trim, dutrim)[8:13], utrim)

        #Create KKT Jacobian
        H = zeros(10,10)

        H[1:u_num, 1:u_num] = Matrix(I, 4, 4)
        H[1:u_num, u_num+1:end] = B'
        H[u_num+1:end, 1:u_num] = B
        
        #Regularize the KKT jacobian
        H = H + Diagonal([ones(u_num); -ones(acc_num)])*ρ

        #first order necessary conditions
        ∇ᵤL = (utrim-u_guess)+B'*λ
        c = dynamics(model, x_trim, utrim)[8:13] #only constrain accelerations

        b = [∇ᵤL ;c]
        
        if(norm(b) < tol)
            break
        end
        
        #calculate newton step
        newton_step = H\-b
        
        #update utrim and lambdas
        utrim = utrim + newton_step[1:4]
        λ = λ+newton_step[5:10]
        
    end

    # Return the trim controls
    return utrim
end

get_trim

In [4]:
# Test that the method finds zero acceleration
@testset "Part a" begin                                                          
    model = YakPlane(UnitQuaternion)
    x0 = SA[
        0,0,0, 
        -6.114599630220157e-8, 0.997155939027903, 0.0, 0.07536599539167085,
        5,0,0,
        0,0,0
    ]

    u_guess = fill(124.0,4)
    utrim = get_trim(model, x0, u_guess)
    
    # Make sure the accelerations are small
    @test norm(dynamics(model, x0, utrim)[8:13]) < 1e-3                          
    @test utrim ≈ load(resfile, "utrim") rtol=1e-3                                
end; # wrap in begin/end to avoid polluting the global state with variables

Test Summary: | Pass  Total
Part a        |    2      2


In [5]:
function lmult(q)
    
    s = q[1]
    v = q[2:4]
    skew = [0 -v[3] v[2]; v[3] 0 -v[1]; -v[2] v[1] 0]
    
    L = [s -v'; v s*I+skew]
    
    #@SMatrix zeros(4,4)
    
    return SMatrix{4,4}(L)
end

lmult (generic function with 1 method)

In [6]:

"""
    state_parts(model::YakPlane)
Give a tuple of indices, `ip,iq,iv,iw` for extracting pieces of the state vector:
* `x[ip]` returns the 3 positions in the world frame
* `x[iq]` returns the 4 elements of the unit quaternion
* `x[iv]` returns the 3 linear velocities in the world frame
* `x[iw]` returns the 3 angular velocities in the body frame
"""
function state_parts(model::YakPlane)
    ip = SA[1,2,3]
    iq = SA[4,5,6,7]
    iv = SA[8,9,10]
    iw = SA[11,12,13]
    return ip, iq, iv, iw
end


function state_error(model::YakPlane, x, x0)
    
    dx = zeros(length(x)-1)
    ip, iq, iv, iw = state_parts(model)
    
    dx[1:3] = x[ip] - x0[ip]
    
    quatdifference = lmult(x0[iq])'*x[iq]
    
    dx[4:6] = quatdifference[2:4]/quatdifference[1] # state is now 12, used Cayley map
    
    dx[7:9] = x[iv] - x0[iv]
    
    dx[10:12] = x[iw] - x0[iw]
    
    return dx
end

state_error (generic function with 1 method)

In [7]:
using BlockDiagonals


"""
    attitude_jacobian(q)

Return the 4x3 attitude Jacobian for the unit quaternion q following the Hamilton convention.
The attitude Jacobian is the Jacobian of `q*ϕ(dq)` with respect to `dq`,
where `ϕ` transforms the infinitessimal rotation `dq` to a unit quaternion.
"""
function attitude_jacobian(q)

    s = q[1]
    v = q[2:4]
    skew = [0 -v[3] v[2]; v[3] 0 -v[1]; -v[2] v[1] 0]
    
    G = [-v'; s*I+skew]
    #return @SMatrix zeros(4,3)
    
    return SMatrix{4,3}(G)
end

"""
    state_error_jacobian(model, x)

Return the Jacobian of the `state_error` function, with respect to the state `x`.
"""
function state_error_jacobian(model, x)
    
    G = attitude_jacobian(x[4:7])

    E = BlockDiagonal([1.0*I(3), G, 1.0*I(6)])
    

    #return E
    
    return SMatrix{length(x), length(x)-1}(E)
end

state_error_jacobian

In [8]:
# Test the attitude and state error Jacobians
@testset "Part b" begin                                          
    model = YakPlane(UnitQuaternion)
    q1 = [0.15923376998393293, 0.26188238416859005, 0.944750125601984, 0.11623004574512731]
    q2 = [-0.511916585919045,  0.03735280673079172, -0.036349007931561726, -0.8574525797457913]
    x1 = [0.989607214000322, 0.51362326582473, 0.3616831284706765, -0.6336197238671922, 0.01573491125215133, -0.7172154648116752, -0.28962119247272433, 0.008365202831213558, 0.5515476660019731, 0.19996988824486706, 0.25027308097388, 0.8555278096133965, 0.9134644393324214]
    x2 = [0.45493888030931307, 0.8214867872560794, 0.587845774939719, 0.0859756782429905, -0.357328766626224, -0.4039767922209037, -0.8376915223630642, 0.274895624769145, 0.40424715886322726, 0.7686052831243393, 0.1958635140022562, 0.33959528953302853, 0.06666659269188946]
    u = [124.17276299136248, 135.60429047063715, 145.9949664404939, 24.18838184023805]
    
    @test lmult(q2)*q1 ≈ load(resfile, "q2q1") rtol=1e-6          
    @test lmult(q1)'q1 ≈ [1,0,0,0] rtol=1e-8                      
    
    dx = state_error(model, x1, x2)
    @test dx ≈ load(resfile, "dx") rtol=1e-6                      

    G0 = attitude_jacobian(q1)
    @test G0 ≈ load(resfile, "G1")                                
    G = state_error_jacobian(model, x1)
    @test G*dx ≈ load(resfile, "Gdx") rtol=1e-6                   
end;

Test Summary: | Pass  Total
Part b        |    5      5


In [9]:

"""
    RigidBodyCost{m,T}

A cost function that penalizes distance of a rigid body state specified by its 
position `p`, attitude `q` (a unit quaternion), linear velocity `v` and angular velocity `ω`.

The vector state use a standard quadratic penalty, and the attitude uses the geodesic distance, weighted by a scalar `w`.
Uses a quadratic penalty on the distance of the `m`-dimensional control vector from a reference control `u_ref`.
"""
struct RigidBodyCost{m,T}
    Qp::Diagonal{T,SVector{3,T}}
    w::T
    Qv::Diagonal{T,SVector{3,T}}
    Qω::Diagonal{T,SVector{3,T}}
    R::Diagonal{T,SVector{m,T}}
    p_ref::SVector{3,T}
    q_ref::SVector{4,T}
    v_ref::SVector{3,T}
    ω_ref::SVector{3,T}
    u_ref::SVector{m,T}
end

"""
    stagecost(cost::RigidBodyCost, x, u)

Calculate the cost for a single knot point for the `RigidBodyCost`. 
TIP: Call `termcost` and add the control cost, to avoid repeated code
"""
function stagecost(cost::RigidBodyCost, x, u)
    # TODO: Calculuate the stage cost
    J_term = termcost(cost, x)
    J = J_term + 0.5*(u-cost.u_ref)'*cost.R*(u-cost.u_ref)
    return J
end

"""
    termcost(cost::RigidBodyCost, x)

Calculate the state-only terminal cost for a `RigidBodyCost`. 
Should use the geodesic distance to compute the error between the 
quaternion in `x` and `cost.q_ref`.
"""
function termcost(cost::RigidBodyCost, x)
    # TODO: Calculate the state-only cost
    J = NaN
    p = x[1:3]
    q = x[4:7]
    v = x[8:10]
    ω = x[11:13]
    J = 0.5*((p-cost.p_ref)'*cost.Qp*(p-cost.p_ref) + (v-cost.v_ref)'*cost.Qv*(v-cost.v_ref)+(ω-cost.ω_ref)'*cost.Qω*(ω-cost.ω_ref))+cost.w*(1-abs(cost.q_ref'*q))
    return J
end

"""
    gradient(cost::RigidBodyCost, x, u)

Calculate the gradient of the cost function with respect to
the states `x` and controls `u`. Should return both as a tuple.
"""
function gradient(cost::RigidBodyCost, x, u)
    # TODO: Calculate the gradients with respect to both x and u
    
    p = x[1:3]
    q = x[4:7]
    v = x[8:10]
    ω = x[11:13]
    
    Qp = Matrix(deepcopy(cost.Qp))
    Qv = Matrix(deepcopy(cost.Qv))
    Qω = Matrix(deepcopy(cost.Qω))
    
    partial_p = Qp*(p-cost.p_ref)
    partial_v = Qv*(v-cost.v_ref)
    partial_ω = Qω*(ω-cost.ω_ref)
    partial_q = vec(cost.w*-sign(cost.q_ref'*q)*cost.q_ref')
    
    grad_x = zero(x)*NaN
    grad_u = zero(u)*NaN
    
    grad_u = cost.R*(u-cost.u_ref)
    
    grad_x = vec([partial_p' partial_q' partial_v' partial_ω'])

    return grad_x, grad_u
end

"""
    hessian(cost::RigidBodyCost, x, u)

Calculate the Hessian of the cost function with respect to both the 
states `x` and controls `u` for a `RigidBodyCost`. 
Should return a tuple of both Hessians
"""
function hessian(cost::RigidBodyCost, x, u)
    # TODO: Calculate Cost Hessian
    Q = @SMatrix ones(length(x), length(x))
    R = @SMatrix ones(length(u), length(u))
    
    q = x[4:7]
    
    hessian_q = zeros(4,4)
    
    Q = [cost.Qp zeros(3, 10); zeros(4,3) hessian_q zeros(4,6); zeros(3,7) cost.Qv zeros(3,3); zeros(3,10) cost.Qω]
    
    R = cost.R
    
    return Q, R
end

"""
    cost(obj, X, U)

Calculate the cost along an entire trajectory using a collection
of cost function types (stored in the iteratable collection `obj`).
"""
function cost(obj, X, U)
    J = zero(eltype(X[1]))
    for k = 1:length(U)
        J += stagecost(obj[k], X[k], U[k])
    end
    J += termcost(obj[end], X[end])
    return J
end

cost

In [10]:
# Test cost and derivatives
@testset "Part c" begin                                                                      
    model = YakPlane(UnitQuaternion)
    x = SA[0.45493888030931307, 0.8214867872560794, 0.587845774939719, 0.0859756782429905, -0.357328766626224, -0.4039767922209037, -0.8376915223630642, 0.274895624769145, 0.40424715886322726, 0.7686052831243393, 0.1958635140022562, 0.33959528953302853, 0.06666659269188946]
    u = SA[124.17276299136248, 135.60429047063715, 145.9949664404939, 24.18838184023805]
    ip,iq,iv,iw = state_parts(model)
    costfun = RigidBodyCost(
        Diagonal(@SVector fill(0.1, 3)),
        10.0, 
        Diagonal(@SVector fill(0.1, 3)),
        Diagonal(@SVector fill(0.1, 3)),
        Diagonal(@SVector fill(1e-3, 4)),
        SA[0,0,0.],
        SA[1,0,0,0.],
        SA[5,0,0.],
        SA[0,0,0.],
        u
    )
    
    J1 = stagecost(costfun, x, u)
    xflip = [x[ip]; -x[iq]; x[iv]; x[iw]]
    @test stagecost(costfun, xflip, u) ≈ J1                                                  
    @test J1 ≈ load(resfile, "J1") rtol=1e-5                                                 
    
    Jterm = termcost(costfun, x)
    @test Jterm ≈ load(resfile, "Jterm") atol=1e-8 rtol=1e-5                                 

    @test ForwardDiff.gradient(x->termcost(costfun,x), x) ≈ gradient(costfun, x, u)[1]       
    @test ForwardDiff.gradient(u->stagecost(costfun,x,u), u) ≈ gradient(costfun, x, u)[2]   
    @test ForwardDiff.hessian(x->termcost(costfun,x), x) ≈ hessian(costfun, x, u)[1]         
    @test ForwardDiff.hessian(u->stagecost(costfun,x,u), u) ≈ hessian(costfun, x, u)[2]      
end;

Test Summary: | Pass  Total
Part c        |    7      7


In [11]:
"""
    iLQRProblem{n,m,L}

Describes a trajectory optimization problem with `n` states, `m` controls, and 
a model of type `L`. 

# Constructor
    Problem(model::L, obj::Vector{<:QuadraticCost{n,m}}, tf, x0, xf) where {n,m,L}

where `tf` is the final time, and `x0` is the initial state. 
"""
struct iLQRProblem{n,m,L,O}
    model::L
    obj::Vector{O}
    N::Int
    tf::Float64
    x0::MVector{n,Float64}
    times::Vector{Float64}
    function iLQRProblem(model::L, obj::Vector{O}, tf, x0) where {L,O}
        n,m = size(model)
        @assert length(x0) == n
        T = length(obj)
        times = range(0, tf, length=T)
        new{n,m,L,O}(model, obj, T, tf, x0, times)
    end
end
Base.size(prob::iLQRProblem{n,m}) where {n,m} = (n,m,prob.N)

In [12]:

"""
    backwardpass!(prob, P, p, K, d, X, U)

Evaluate the iLQR backward pass at state and control trajectories `X` and `U`, 
storing the cost-to-go expansion in `P` and `p` and the gains in `K` and `d`.

Should return ΔJ, expected cost reduction.
"""
function backwardpass!(prob::iLQRProblem{n,m}, P, p, K, d, X, U; 
        β=1e-6
    ) where {n,m}
    N = prob.N
    ΔJ = 0.0
    model = prob.model
    
    iq = state_parts(prob.model)[2]
    Iq = Diagonal(SA[0,0,0,1,1,1, 0,0,0, 0,0,0])

    # Calculate gradient and Hessian at terminal state
    Q, = hessian(prob.obj[N], X[N], 0*U[1])
    q, = gradient(prob.obj[N], X[N], 0*U[1])
    
    #Calculate the terminal cost-to-go, accounting for the group structure of the 3D rotation
    
    E = state_error_jacobian(model, X[N])
    
    p[N] = (q'*E)' #jacobian times jacobian. then transpose to make it the gradient

    P[N] = E'*Q*E - Iq*(dot(q[iq], X[N][iq]))
    
    #Backward Pass
    for k = (N-1):-1:1 

        # Cost Expansion
        Q,R =  hessian(prob.obj[k], X[k], U[k])
        q,r = gradient(prob.obj[k], X[k], U[k])
        
        E_plus1 = state_error_jacobian(model, X[k+1])
        E = state_error_jacobian(model, X[k])
        
        # Dynamics derivatives
        dt = prob.times[k+1] - prob.times[k]
        A,B = discrete_jacobian(model, X[k], U[k], prob.times[k], dt)

        # Account for the group structure of 3D rotations using the attitude Jacobian (via state_error_jacobian)
        
        q = (q'*E)'
        Q = E'*Q*E - Iq*(dot(q[iq], X[k][iq]))
        
        A = E_plus1'*A*E
        B = E_plus1'*B
        
        gx = q + A'*p[k+1]
        gu = r + B'*p[k+1]
    
        Gxx = Q + A'*P[k+1]*A
        Guu = R + B'*P[k+1]*B
        Gux = B'*P[k+1]*A
        
        # Regularization
        Gux_reg = SMatrix{m,n-1}(Gux)
        Guu_reg = SMatrix{m,m}(Guu + β*Diagonal(@SVector ones(m)))
        
        # Calculate Gains
        F = cholesky(Symmetric(Guu_reg))
        d[k] .= (F\gu)
        K[k] .= (F\Gux_reg)
    
        # Cost-to-go Recurrence
        p[k] .= gx - K[k]'*gu + K[k]'*Guu*d[k] - Gux'*d[k]
        P[k] .= Gxx + K[k]'*Guu*K[k] - Gux'*K[k] - K[k]'*Gux
        ΔJ += gu'*d[k]
   
    end
    return ΔJ
end

backwardpass!

In [13]:
"""
    forwardpass!(prob, X, U, K, d, ΔJ, J)

Evaluate the iLQR forward pass at state and control trajectories `X` and `U`, using
the gains `K` and `d` to simulate the system forward. The new cost should be less than 
the current cost `J` together with the expected cost decrease `ΔJ`.

Should return the new cost `Jn` and the step length `α`.
"""
function forwardpass!(prob::iLQRProblem{n,m}, X, U, K, d, ΔJ, J,
        Xbar = deepcopy(X), Ubar = deepcopy(U);
        max_iters=10,
    ) where {n,m}
    N = prob.N
    model = prob.model

    Jn = J
    α = 0.0
    
    # Line Search# 
"""
    forwardpass!(prob, X, U, K, d, ΔJ, J)

Evaluate the iLQR forward pass at state and control trajectories `X` and `U`, using
the gains `K` and `d` to simulate the system forward. The new cost should be less than 
the current cost `J` together with the expected cost decrease `ΔJ`.

Should return the new cost `Jn` and the step length `α`.
"""
function forwardpass!(prob::iLQRProblem{n,m}, X, U, K, d, ΔJ, J,
        Xbar = deepcopy(X), Ubar = deepcopy(U);
        max_iters=10,
    ) where {n,m}
    N = prob.N
    model = prob.model

    Jn = J
    α = 0.0
    
    # Line Search
    Xbar[1] = X[1]
    α = 1.0
    Jn = Inf
    for i = 1:max_iters
        
        # Forward Rollout
        for k = 1:(N-1)
            t = prob.times[k]
            dt = prob.times[k+1] - prob.times[k]
        
            # Apply the local feedback control on the error state
            δx = state_error(model, Xbar[k], X[k]) 
            
            Ubar[k] = U[k] - α*d[k] - (K[k] * δx)
            
            Xbar[k+1] = discrete_dynamics(model, Xbar[k], Ubar[k], t, dt)
            
            #@show any(isnan.(Xbar[k+1]))
        
        end
        
        # Calculate the new cost
        Jn = cost(prob.obj, Xbar, Ubar)

        # Check Armijo condition
        if Jn <= J - 1e-2*α*ΔJ
            break
        else
            # Backtrack
            α *= 0.5  
        end
        if i == max_iters 
            α = 0
        end
    end
    
    # Accept direction
    for k = 1:N-1
        X[k] = Xbar[k]
        U[k] = Ubar[k]
    end
    X[N] = Xbar[N]
    
    return Jn, α
end
    Xbar[1] = X[1]
    α = 1.0
    Jn = Inf
    for i = 1:max_iters
        
        # Forward Rollout
        for k = 1:(N-1)
            t = prob.times[k]
            dt = prob.times[k+1] - prob.times[k]
        
            # Apply the local feedback control on the error state
            δx = state_error(model, Xbar[k], X[k]) 
            
            Ubar[k] = U[k] - α*d[k] - (K[k] * δx)
            
            Xbar[k+1] = discrete_dynamics(model, Xbar[k], Ubar[k], t, dt)
            
            #@show any(isnan.(Xbar[k+1]))
        
        end
        
        # Calculate the new cost
        Jn = cost(prob.obj, Xbar, Ubar)

        # Check Armijo condition
        if Jn <= J - 1e-2*α*ΔJ
            break
        else
            # Backtrack
            α *= 0.5  
        end
        if i == max_iters 
            α = 0
        end
    end
    
    # Accept direction
    for k = 1:N-1
        X[k] = Xbar[k]
        U[k] = Ubar[k]
    end
    X[N] = Xbar[N]
    
    return Jn, α
end

forwardpass!

In [14]:
"""
    solve_ilqr(prob, X, U; kwargs...)

Solve the trajectory optimization problem specified by `prob` using iterative LQR.
Returns the optimized state and control trajectories, as well as the local control gains,
`K` and `d`.

Should return the optimized state and control trajectories `X` and `U`, and the 
list of feedback gains `K` and cost-to-go hessians `P`.
"""
function solve_ilqr(prob::iLQRProblem{n,m}, X0, U0; 
        iters=100,     # max iterations
        ls_iters=10,   # max line search iterations
        reg_min=1e-6,  # minimum regularizatio for the backwardpass
        verbose=0,     # print verbosity
        eps=1e-5,      # termination tolerance
        eps_ddp=eps    # tolerance to switch to ddp
    ) where {n,m}
    t_start = time_ns()
    Nx,Nu,Nt = size(prob)

    # Initialization
    # NOTE: some of these sizes have changed!
    N = prob.N
    p = [zeros(n-1) for k = 1:N]              # ctg gradient
    P = [zeros(n-1,n-1) for k = 1:N]          # ctg hessian
    d = [zeros(m) for k = 1:N-1]              # feedforward gains
    K = [zeros(m,n-1) for k = 1:N-1]          # feedback gains
    Xbar = [@SVector zeros(n) for k = 1:N]    # line search trajectory
    Ubar = [@SVector zeros(m) for k = 1:N-1]  # line search trajectory
    ΔJ = 0.0

    # Don't modify the trajectories that are passed in
    X = deepcopy(X0)
    U = deepcopy(U0)

    # Initial cost
    J = cost(prob.obj, X, U)
    
    # Initialize parameters
    Jn = Inf
    iter = 0
    tol = 1.0
    β = reg_min
    while tol > eps 
        iter += 1
        
        # Backward Pass
        ΔJ, = backwardpass!(prob, P, p, K, d, X, U, β=β)

        # Forward Pass
        Jn, α = forwardpass!(prob, X, U, K, d, ΔJ, J, Xbar, Ubar, max_iters=ls_iters)

        if α === zero(α) 
            β = max(β*10, 1.0)
            # β *= 10 
        else 
            β = max(β/2, reg_min)
        end

        # Update parameters
        tol = maximum(norm.(d, Inf))
        β = max(0.9*β, reg_min)

        # Output
        if verbose > 0
            @printf("Iter: %3d, Cost: % 6.2f → % 6.2f (% 7.2e), res: % .2e, β= %.2e, α = %.3f, ΔJ = %.3e\n",
                iter, J, Jn, J-Jn, tol, β, α, ΔJ
            )
        end
        J = Jn

        if iter >= iters
            @warn "Reached max iterations"
            break
        end

    end
    println("Total Time: ", (time_ns() - t_start)*1e-6, " ms")
    return X,U,K,P
end

solve_ilqr

In [15]:
using RobotDynamics: RBState
function YakProblems(;
        N = 101,
        vecstate=false,
        scenario=:barrellroll, 
        heading=0.0,  # deg
        Qpos=1.0,
        kwargs...
    )
    model = YakPlane(UnitQuaternion)

    n,m = size(model)
    ip,iq,iv,iw = state_parts(model)

    # Discretization
    tf = 1.25
    if scenario == :fullloop 
        # Double the time, keeping the same discretization
        tf *= 2
        N = (N-1)*2 + 1
    end
    dt = tf/(N-1)

    # Forward Velocity 
    vel = 5.0

    if scenario ∈ (:halfloop, :fullloop) 
        ey = @SVector [0,1,0.]

        # Heading
        dq = expm(SA[0,0,1]*deg2rad(heading))

        # Initial state
        p0 = MRP(0.997156, 0., 0.075366) # initial orientation (level flight)
        x0 = RD.build_state(model, [-3,0,1.5], p0, [vel,0,0], [0,0,0])

        # Climb
        pm = expm(SA[1,0,0]*deg2rad(180))*expm(SA[0,1,0]*deg2rad(90))
        xm = RD.build_state(model, [0,0,3.], pm, pm * [vel,0,0.], [0,0,0])

        # Top of loop
        pf = MRP(0., -0.0366076, 0.) * dq # final orientation (upside down)
        xf = RD.build_state(model, dq*[3,0,6.], pf, pf * [vel,0,0.], [0,0,0])
        pf2 = RotZ(deg2rad(heading-180))

        # Dive
        xm2 = RD.build_state(model, [-3,3,4.], pm * RotY(pi), [0,0,-vel], [0,0,0])

        # Terminal state
        xf2 = RD.build_state(model, [0,4,1.5], p0, [vel,0,0], [0,0,0])

        t_flat = 5 / (xf[2] - xf[1])
        N_flat = Int(round(t_flat/dt))

        # Xref trajectory
        x̄0 = RBState(model, x0)
        x̄m = RBState(model, xm)
        x̄m2 = RBState(model, xm2)
        x̄f = RBState(model, xf)
        x̄f2 = RBState(model, xf2)
        Xref = map(1:N) do k
            t = (k-1)/(N-1)
            Nmid = N ÷ 4
            if scenario == :fullloop
                if k < Nmid
                    x1 = x̄0
                    x2 = x̄m
                    t = (k-1)/Nmid
                elseif k < 2Nmid
                    t = (k-Nmid)/Nmid
                    x1 = x̄m
                    x2 = x̄f 
                elseif k < 3Nmid
                    t = (k-2Nmid)/Nmid
                    x1 = x̄f
                    x2 = x̄m2
                else
                    t = (k-3Nmid)/Nmid
                    x1 = x̄m2
                    x2 = x̄f2
                end
            else
                if k < 2Nmid
                    t = (k-1)/2Nmid
                    x1 = x̄0
                    x2 = x̄m
                else
                    t = (k-2Nmid)/(2Nmid+1)
                    x1 = x̄m
                    x2 = x̄f
                end
            end
            RBState(
                x1.r + (x2.r - x1.r)*t,
                slerp(x1.q, x2.q, t),
                x1.v + (x2.v - x1.v)*t,
                SA[0,pi/1.25,0]
            )
        end
    else
        throw(ArgumentError("$scenario isn't a known scenario"))
    end

    # Get trim condition
    utrim = get_trim(model, x0, fill(124.0, 4))

    # Objective
    Qf_diag = RD.fill_state(model, 10, 500*0, 100, 100.)
    Q_diag = RD.fill_state(model, Qpos*0.1, 0.1*0, 0.1, 1.1)
    R = Diagonal(@SVector fill(1e-3,4))
    costs = map(1:N-1) do k
        RigidBodyCost(
            Diagonal(Q_diag[ip])*dt,
            10.0*dt, 
            Diagonal(Q_diag[iv])*dt,
            Diagonal(Q_diag[iw])*dt,
            R*dt,
            Xref[k][ip],
            Xref[k][iq],
            Xref[k][iv],
            Xref[k][iw],
            SVector{4,Float64}(utrim)
        )
    end
    costterm = RigidBodyCost(
            Diagonal(Qf_diag[ip]),
            200.0, 
            Diagonal(Qf_diag[iv]),
            Diagonal(Qf_diag[iw]),
            R*0,
            Xref[N][ip],
            Xref[N][iq],
            Xref[N][iv],
            Xref[N][iw],
            SVector{4,Float64}(utrim)
        )
    push!(costs, costterm)

    # Build problem
    prob = iLQRProblem(model, costs, tf, x0)

    return prob, Xref, utrim
end

YakProblems (generic function with 1 method)

In [16]:
prob_half, Xref_half = YakProblems(costfun=:QuatLQR, scenario=:halfloop, heading=130)

(iLQRProblem{13, 4, YakPlane{UnitQuaternion, Float64}, RigidBodyCost{4, Float64}}(YakPlane{UnitQuaternion, Float64}
  g: Float64 9.81
  rho: Float64 1.2
  m: Float64 0.075
  Jx: Float64 0.00048944
  Jy: Float64 0.00063778
  Jz: Float64 0.00079509
  J: Diagonal{Float64, SVector{3, Float64}}
  Jinv: Diagonal{Float64, SVector{3, Float64}}
  Jm: Float64 3.660416666666667e-6
  b: Float64 0.45
  l_in: Float64 0.06
  cr: Float64 0.135
  ct: Float64 0.08
  cm: Float64 0.10750000000000001
  S: Float64 0.04837500000000001
  S_in: Float64 0.0162
  S_out: Float64 0.03217500000000001
  Rt: Float64 0.5925925925925926
  r_ail: Float64 0.10290697674418604
  ep_ail: Float64 0.63
  trim_ail: Float64 106.0
  g_ail: Float64 0.002617993877991494
  b_elev: Float64 0.16
  cr_elev: Float64 0.06
  ct_elev: Float64 0.04
  cm_elev: Float64 0.05
  S_elev: Float64 0.008
  Ra_elev: Float64 3.2
  r_elev: Float64 0.22
  ep_elev: Float64 0.88
  trim_elev: Float64 106.0
  g_elev: Float64 0.0034906585039886587
  b_rud: 

In [17]:
vis = Visualizer()
TrajOptPlots.set_mesh!(vis, YakPlane(UnitQuaternion), color=colorant"blue")
render(vis)  # this may take a while, it's a detailed model

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
└ http://127.0.0.1:8700


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))))), Set{Any}(), ip"127.0.0.1", 8700))

In [18]:
visualize!(vis, prob_half.model, prob_half.tf, Xref_half)

In [19]:
X0 = [SVector(prob_half.x0) for k = 1:prob_half.N]      # remember this needs to be dynamically feasible!

utrim = get_trim(prob_half.model, X0[1], fill(124.0, 4))

#println(utrim)

U0 = [copy(utrim) for k = 1:prob_half.N-1] #fill u0

times = prob_half.times 

for k = 1:prob_half.N-1
    
    t = times[k]
    dt = prob_half.times[k+1] - prob_half.times[k]
    X0[k+1] = discrete_dynamics(prob_half.model, X0[k], utrim, t, dt)
    #println(X0[k+1])
end
visualize!(vis, prob_half.model, prob_half.tf, X0)

In [20]:
Xhalf, Uhalf = solve_ilqr(prob_half, X0, U0, verbose=1, eps=1e-4, reg_min=1e-6)
visualize!(vis, prob_half.model, prob_half.tf, Xhalf)

Iter:   1, Cost:  4878.39 →  4621.95 ( 2.56e+02), res:  5.81e+03, β= 1.00e-06, α = 0.062, ΔJ = 9.578e+03
Iter:   2, Cost:  4621.95 →  4296.81 ( 3.25e+02), res:  7.39e+03, β= 1.00e-06, α = 0.031, ΔJ = 8.863e+03
Iter:   3, Cost:  4296.81 →  3742.09 ( 5.55e+02), res:  5.41e+03, β= 1.00e-06, α = 0.062, ΔJ = 8.372e+03
Iter:   4, Cost:  3742.09 →  2828.06 ( 9.14e+02), res:  6.71e+03, β= 1.00e-06, α = 0.125, ΔJ = 7.357e+03
Iter:   5, Cost:  2828.06 →  1666.31 ( 1.16e+03), res:  5.23e+03, β= 1.00e-06, α = 0.500, ΔJ = 5.603e+03
Iter:   6, Cost:  1666.31 →  249.73 ( 1.42e+03), res:  2.15e+03, β= 1.00e-06, α = 1.000, ΔJ = 3.260e+03
Iter:   7, Cost:  249.73 →  103.16 ( 1.47e+02), res:  9.18e+02, β= 1.00e-06, α = 0.500, ΔJ = 4.800e+02
Iter:   8, Cost:  103.16 →   8.20 ( 9.50e+01), res:  4.93e+02, β= 1.00e-06, α = 1.000, ΔJ = 1.907e+02
Iter:   9, Cost:   8.20 →   6.74 ( 1.47e+00), res:  1.33e+02, β= 1.00e-06, α = 0.500, ΔJ = 3.955e+00
Iter:  10, Cost:   6.74 →   6.22 ( 5.17e-01), res:  7.12e+01, β= 

In [21]:
#new test
@testset "Part d" begin                                                                 
    let prob = prob_half
        n,m,N = size(prob)
        p = [zeros(n-1) for k = 1:N]              # ctg gradient
        P = [zeros(n-1,n-1) for k = 1:N]          # ctg hessian
        d = [zeros(m) for k = 1:N-1]              # feedforward gains
        K = [zeros(m,n-1) for k = 1:N-1]          # feedback gains
        Xbar = [@SVector zeros(n) for k = 1:N]    # line search trajectory
        Ubar = [@SVector zeros(m) for k = 1:N-1]  # line search trajectory

        X = deepcopy(X0)
        U = deepcopy(U0)

        # Initial cost
        J = cost(prob.obj, X, U)
        @test J ≈ load(resfile, "J0")                                                 
        
        # Backward Pass
        ΔJ, = backwardpass!(prob, P, p, K, d, X, U, β=1e-8)
        @test ΔJ ≈ load(resfile, "dJ") rtol=1e-3                                        
        @test norm(d,Inf) ≈ load(resfile, "d0") rtol=1e-3                               

        # Forward Pass
        Jn, α = forwardpass!(prob, X, U, K, d, ΔJ, J, Xbar, Ubar, max_iters=10)
        @test Jn < J                                                                    
        @test (J-Jn) / ΔJ ≈ 0.025 atol=1e-2                                             
        @test Jn ≈ load(resfile, "Jn") rtol=1e-1                                        
    end
    
end;

Test Summary: | Pass  Total
Part d        |    6      6


In [22]:
visualize!(vis, prob_half.model, prob_half.tf, Xhalf)

In [23]:
prob, Xref_full = YakProblems(costfun=:QuatLQR, scenario=:fullloop, heading=130, Qpos=100)
render(vis)

MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), UInt8[0x83, 0xa7, 0x6f, 0x70, 0x74, 0x69, 0x6f, 0x6e, 0x73, 0x82  …  0x5f, 0x61, 0x6e, 0x69, 0x6d, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))))), Set(Any[WebSocket(server, CONNECTED)]), ip"127.0.0.1", 8700))

In [24]:
visualize!(vis, prob.model, prob.tf, Xref_full)

In [25]:
X0 = [SVector(prob.x0) for k = 1:prob.N]

controls1 = copy(Uhalf)
controls2 = reverse(controls1)
U0 = [controls1; controls2]

times = prob.times 

for k = 1:prob.N-1
    
    t = times[k]
    dt = times[k+1] - times[k]
    X0[k+1] = discrete_dynamics(prob.model, X0[k], U0[k], t, dt)
end

In [26]:
visualize!(vis, prob.model, prob.tf, X0)

In [27]:
Xfull, Ufull = solve_ilqr(prob, X0, U0, verbose=1, eps=1e-2, reg_min=1e-8, iters=500)

Iter:   1, Cost:  9892.82 →  5236.26 ( 4.66e+03), res:  1.00e+04, β= 1.00e-08, α = 0.250, ΔJ = 1.973e+04
Iter:   2, Cost:  5236.26 →  3970.20 ( 1.27e+03), res:  7.22e+03, β= 1.00e-08, α = 0.125, ΔJ = 1.043e+04
Iter:   3, Cost:  3970.20 →  3482.81 ( 4.87e+02), res:  6.06e+03, β= 1.00e-08, α = 0.062, ΔJ = 7.896e+03
Iter:   4, Cost:  3482.81 →  2999.30 ( 4.84e+02), res:  5.53e+03, β= 1.00e-08, α = 0.062, ΔJ = 6.922e+03
Iter:   5, Cost:  2999.30 →  2636.90 ( 3.62e+02), res:  6.09e+03, β= 1.00e-08, α = 0.062, ΔJ = 5.953e+03
Iter:   6, Cost:  2636.90 →  2296.77 ( 3.40e+02), res:  5.59e+03, β= 1.00e-08, α = 0.062, ΔJ = 5.228e+03
Iter:   7, Cost:  2296.77 →  1997.60 ( 2.99e+02), res:  4.41e+03, β= 1.00e-08, α = 0.062, ΔJ = 4.549e+03
Iter:   8, Cost:  1997.60 →  1759.06 ( 2.39e+02), res:  4.73e+03, β= 1.00e-08, α = 0.062, ΔJ = 3.948e+03
Iter:   9, Cost:  1759.06 →  1543.15 ( 2.16e+02), res:  4.36e+03, β= 1.00e-08, α = 0.062, ΔJ = 3.471e+03
Iter:  10, Cost:  1543.15 →  1285.20 ( 2.58e+02), res: 

Iter:  83, Cost:  14.44 →  14.44 (-3.12e-03), res:  5.03e+01, β= 9.00e-01, α = 0.000, ΔJ = 1.920e+00
Iter:  84, Cost:  14.44 →  14.43 ( 1.11e-02), res:  2.91e-02, β= 4.05e-01, α = 1.000, ΔJ = 1.141e-02
Iter:  85, Cost:  14.43 →  14.42 ( 9.18e-03), res:  5.02e-02, β= 1.82e-01, α = 1.000, ΔJ = 9.311e-03
Iter:  86, Cost:  14.42 →  14.41 ( 1.00e-02), res:  8.64e-02, β= 8.20e-02, α = 1.000, ΔJ = 1.019e-02
Iter:  87, Cost:  14.41 →  14.40 ( 1.06e-02), res:  1.36e-01, β= 3.69e-02, α = 1.000, ΔJ = 1.221e-02
Iter:  88, Cost:  14.40 →  14.39 ( 1.16e-02), res:  2.07e-01, β= 1.66e-02, α = 1.000, ΔJ = 1.226e-02
Iter:  89, Cost:  14.39 →  14.39 ( 2.06e-03), res:  5.36e-01, β= 7.47e-03, α = 1.000, ΔJ = 2.405e-02
Iter:  90, Cost:  14.39 →  14.39 ( 1.84e-03), res:  9.09e-01, β= 3.36e-03, α = 0.031, ΔJ = 1.199e-01
Iter:  91, Cost:  14.39 →  14.37 ( 2.10e-02), res:  2.92e+00, β= 1.51e-03, α = 1.000, ΔJ = 9.047e-02
Iter:  92, Cost:  14.37 →  14.36 ( 4.93e-03), res:  4.73e+00, β= 6.81e-04, α = 0.062, ΔJ = 

Iter: 165, Cost:  14.09 →  14.08 ( 4.59e-03), res:  1.51e-01, β= 1.66e-02, α = 1.000, ΔJ = 5.913e-03
Iter: 166, Cost:  14.08 →  14.08 ( 1.98e-03), res:  3.23e-01, β= 7.47e-03, α = 1.000, ΔJ = 1.113e-02
Iter: 167, Cost:  14.08 →  14.08 ( 2.09e-03), res:  7.52e-01, β= 3.36e-03, α = 0.125, ΔJ = 2.307e-02
Iter: 168, Cost:  14.08 →  14.09 (-6.97e-03), res:  1.53e+00, β= 9.00e-01, α = 0.000, ΔJ = 7.858e-02
Iter: 169, Cost:  14.09 →  14.09 ( 3.84e-04), res:  1.65e-02, β= 4.05e-01, α = 0.250, ΔJ = 1.043e-03
Iter: 170, Cost:  14.09 →  14.09 ( 2.79e-05), res:  3.04e-02, β= 1.82e-01, α = 0.016, ΔJ = 1.344e-03
Iter: 171, Cost:  14.09 →  14.09 ( 5.65e-04), res:  5.63e-02, β= 8.20e-02, α = 1.000, ΔJ = 2.225e-03
Iter: 172, Cost:  14.09 →  14.08 ( 3.11e-03), res:  1.46e-01, β= 3.69e-02, α = 1.000, ΔJ = 1.096e-02
Iter: 173, Cost:  14.08 →  14.08 ( 2.37e-03), res:  1.78e-01, β= 1.66e-02, α = 1.000, ΔJ = 5.635e-03
Iter: 174, Cost:  14.08 →  14.07 ( 9.15e-03), res:  4.89e-01, β= 7.47e-03, α = 1.000, ΔJ = 

(SVector{13, Float64}[[-3.0, 0.0, 1.5, -6.114599630220157e-8, 0.997155939027903, 0.0, 0.07536599539167085, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-2.93688416764603, -5.3605693938757286e-5, 1.499744830074555, 0.0019925345999836663, 0.9966316181190773, 0.0003003117880073021, 0.08200130468272027, 5.097989406023027, -0.008361011047421596, -0.03234917703581628, -0.6104590845463345, 2.00947224103706, -0.11460587132466316], [-2.872621033927447, -0.00019670557524775818, 1.499622024264281, 0.006711114574897979, 0.9953436086696529, 0.0006828642470383737, 0.09617048413285548, 5.183048147382146, -0.014236478504676128, 0.018114990352233296, -0.8784759498131753, 2.4959210385553092, -0.11627352347395684], [-2.807344669671356, -0.0004027782223207458, 1.5003720392196114, 0.012574420267527012, 0.9936356209250571, 0.0009388171415345263, 0.11194843212052094, 5.259791452193237, -0.018293503894084837, 0.10580844796558357, -0.9856691459141299, 2.55841272649197, -0.11191940939177875], [-2.7411563412361346, -0.000643

In [28]:
@testset "Part f" begin                                                                    
    @test norm(RBState(prob.model,Xfull[end]) ⊖ RBState(prob.model,Xref_full[end])) < 0.5
    @test cost(prob.obj, Xfull, Ufull) < 14.5
    let prob = prob, X=Xfull, U=Ufull
        n,m,N = size(prob)
        p = [zeros(n-1) for k = 1:N]              # ctg gradient
        P = [zeros(n-1,n-1) for k = 1:N]          # ctg hessian
        d = [zeros(m) for k = 1:N-1]              # feedforward gains
        K = [zeros(m,n-1) for k = 1:N-1]          # feedback gains
        ΔJ, = backwardpass!(prob, P, p, K, d, X, U, β=1.0)
        @test ΔJ < 1.0                                                                      
        @test norm(d) < 1.0                                                                 
    end
end;

Test Summary: | Pass  Total
Part f        |    4      4


In [29]:
render(vis)

MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), UInt8[0x83, 0xa7, 0x6f, 0x70, 0x74, 0x69, 0x6f, 0x6e, 0x73, 0x82  …  0x5f, 0x61, 0x6e, 0x69, 0x6d, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))))), Set(Any[WebSocket(server, CONNECTED), WebSocket(server, CONNECTED)]), ip"127.0.0.1", 8700))

In [30]:
visualize!(vis, prob.model, prob.tf, Xfull)

In [31]:
using DelimitedFiles
writedlm( "iLQR_Result.csv",  Xfull, ',')
writedlm("loop_ref.csv", Xref_full , ',')

In [32]:
using RobotDynamics: RBState
function YakProblems_2(;
        N = 101,
        vecstate=false,
        scenario=:barrellroll, 
        heading=0.0,  # deg
        Qpos=1.0,
        kwargs...
    )
    model = YakPlane(UnitQuaternion)

    n,m = size(model)
    ip,iq,iv,iw = state_parts(model)

    # Discretization
    tf = 1.25
    if scenario == :fullloop 
        # Double the time, keeping the same discretization
        tf *= 2
        N = (N-1)*2 + 1
    end
    dt = tf/(N-1)

    # Forward Velocity 
    vel = 5.0

    if scenario ∈ (:halfloop, :fullloop) 
        ey = @SVector [0,1,0.]

         # Heading
        dq = expm(SA[0,0,1]*deg2rad(heading))
        dq2 = expm(SA[0,0,1]*deg2rad(180))

        # Initial state
        p0 = MRP(0.997156, 0., 0.075366) # initial orientation (level flight)
        x0 = RD.build_state(model, [-3,0,1.5], p0, [vel,0,0], [0,0,0])

        # Climb
        pm = expm(SA[1,0,0]*deg2rad(180))*expm(SA[0,1,0]*deg2rad(90))
        xm = RD.build_state(model, [0,0,3.], pm, pm * [vel,0,0.], [0,0,0])

        # Top of loop
        pf = MRP(0., -0.0366076, 0.)RotZ(pi)# final orientation (upside down)
        xf = RD.build_state(model, [-2,0,6.], pf, [vel,0,0.], [0,0,0])
        pf2 = RotZ(deg2rad(heading-180))

        # Dive
        xm2 = RD.build_state(model, [-5,0.5,4.], p0*RotZ(pi) , [vel,0,0], [0,0,0])

        # Terminal state
        xf2 = RD.build_state(model, [-20,2,3], p0*RotZ(pi), [vel,0,0], [0,0,0])

        t_flat = 5 / (xf[2] - xf[1])
        N_flat = Int(round(t_flat/dt))

        # Xref trajectory
        x̄0 = RBState(model, x0)
        x̄m = RBState(model, xm)
        x̄m2 = RBState(model, xm2)
        x̄f = RBState(model, xf)
        x̄f2 = RBState(model, xf2)
        Xref = map(1:N) do k
            t = (k-1)/(N-1)
            Nmid = N ÷ 4
            if scenario == :fullloop
                if k < Nmid
                    x1 = x̄0
                    x2 = x̄m
                    t = (k-1)/Nmid
                elseif k < 2Nmid
                    t = (k-Nmid)/Nmid
                    x1 = x̄m
                    x2 = x̄f 
                elseif k < 3Nmid
                    t = (k-2Nmid)/Nmid
                    x1 = x̄f
                    x2 = x̄m2
                else
                    t = (k-3Nmid)/Nmid
                    x1 = x̄m2
                    x2 = x̄f2
                end
            else
                if k < 2Nmid
                    t = (k-1)/2Nmid
                    x1 = x̄0
                    x2 = x̄m
                else
                    t = (k-2Nmid)/(2Nmid+1)
                    x1 = x̄m
                    x2 = x̄f
                end
            end
            RBState(
                x1.r + (x2.r - x1.r)*t,
                slerp(x1.q, x2.q, t),
                x1.v + (x2.v - x1.v)*t,
                SA[0,pi/1.25,0]
            )
        end
    else
        throw(ArgumentError("$scenario isn't a known scenario"))
    end

    # Get trim condition
    utrim = get_trim(model, x0, fill(124.0, 4))

    # Objective
    Qf_diag = RD.fill_state(model, 10, 500*0, 100, 100.)
    Q_diag = RD.fill_state(model, Qpos*0.1, 0.1*0, 0.1, 1.1)
    R = Diagonal(@SVector fill(1e-3,4))
    costs = map(1:N-1) do k
        RigidBodyCost(
            Diagonal(Q_diag[ip])*dt,
            10.0*dt, 
            Diagonal(Q_diag[iv])*dt,
            Diagonal(Q_diag[iw])*dt,
            R*dt,
            Xref[k][ip],
            Xref[k][iq],
            Xref[k][iv],
            Xref[k][iw],
            SVector{4,Float64}(utrim)
        )
    end
    costterm = RigidBodyCost(
            Diagonal(Qf_diag[ip]),
            200.0, 
            Diagonal(Qf_diag[iv]),
            Diagonal(Qf_diag[iw]),
            R*0,
            Xref[N][ip],
            Xref[N][iq],
            Xref[N][iv],
            Xref[N][iw],
            SVector{4,Float64}(utrim)
        )
    push!(costs, costterm)

    # Build problem
    prob = iLQRProblem(model, costs, tf, x0)

    return prob, Xref, utrim
end

YakProblems_2 (generic function with 1 method)

In [33]:
prob, Xref_immelmann = YakProblems_2(costfun=:QuatLQR, scenario=:fullloop, heading=130, Qpos=100)

(iLQRProblem{13, 4, YakPlane{UnitQuaternion, Float64}, RigidBodyCost{4, Float64}}(YakPlane{UnitQuaternion, Float64}
  g: Float64 9.81
  rho: Float64 1.2
  m: Float64 0.075
  Jx: Float64 0.00048944
  Jy: Float64 0.00063778
  Jz: Float64 0.00079509
  J: Diagonal{Float64, SVector{3, Float64}}
  Jinv: Diagonal{Float64, SVector{3, Float64}}
  Jm: Float64 3.660416666666667e-6
  b: Float64 0.45
  l_in: Float64 0.06
  cr: Float64 0.135
  ct: Float64 0.08
  cm: Float64 0.10750000000000001
  S: Float64 0.04837500000000001
  S_in: Float64 0.0162
  S_out: Float64 0.03217500000000001
  Rt: Float64 0.5925925925925926
  r_ail: Float64 0.10290697674418604
  ep_ail: Float64 0.63
  trim_ail: Float64 106.0
  g_ail: Float64 0.002617993877991494
  b_elev: Float64 0.16
  cr_elev: Float64 0.06
  ct_elev: Float64 0.04
  cm_elev: Float64 0.05
  S_elev: Float64 0.008
  Ra_elev: Float64 3.2
  r_elev: Float64 0.22
  ep_elev: Float64 0.88
  trim_elev: Float64 106.0
  g_elev: Float64 0.0034906585039886587
  b_rud: 

In [34]:
render(vis)

MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), UInt8[0x83, 0xa7, 0x6f, 0x70, 0x74, 0x69, 0x6f, 0x6e, 0x73, 0x82  …  0x5f, 0x61, 0x6e, 0x69, 0x6d, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))))), Set(Any[WebSocket(server, CONNECTED), WebSocket(server, CONNECTED), WebSocket(server, CONNECTED)]), ip"127.0.0.1", 8700))

In [35]:
visualize!(vis, prob.model, prob.tf, Xref_immelmann)

In [36]:
Ximm, Uimm = solve_ilqr(prob, Xfull, Ufull, verbose=1, eps=1e-2, reg_min=1e-8, iters=500)

Iter:   1, Cost:  2872.37 →  2827.27 ( 4.51e+01), res:  2.05e+03, β= 1.00e-08, α = 0.008, ΔJ = 6.001e+03
Iter:   2, Cost:  2827.27 →  2700.71 ( 1.27e+02), res:  2.01e+03, β= 1.00e-08, α = 0.031, ΔJ = 5.434e+03
Iter:   3, Cost:  2700.71 →  2635.41 ( 6.53e+01), res:  2.05e+03, β= 1.00e-08, α = 0.016, ΔJ = 4.218e+03
Iter:   4, Cost:  2635.41 →  2531.24 ( 1.04e+02), res:  1.95e+03, β= 1.00e-08, α = 0.031, ΔJ = 4.123e+03
Iter:   5, Cost:  2531.24 →  2423.34 ( 1.08e+02), res:  2.50e+03, β= 1.00e-08, α = 0.031, ΔJ = 3.825e+03
Iter:   6, Cost:  2423.34 →  2352.03 ( 7.13e+01), res:  2.37e+03, β= 1.00e-08, α = 0.031, ΔJ = 3.779e+03
Iter:   7, Cost:  2352.03 →  2305.34 ( 4.67e+01), res:  2.00e+03, β= 1.00e-08, α = 0.016, ΔJ = 3.514e+03
Iter:   8, Cost:  2305.34 →  2133.71 ( 1.72e+02), res:  2.01e+03, β= 1.00e-08, α = 0.062, ΔJ = 3.248e+03
Iter:   9, Cost:  2133.71 →  2072.54 ( 6.12e+01), res:  1.64e+03, β= 1.00e-08, α = 0.031, ΔJ = 3.115e+03
Iter:  10, Cost:  2072.54 →  2004.61 ( 6.79e+01), res: 

Iter:  81, Cost:  604.24 →  603.36 ( 8.80e-01), res:  1.37e+00, β= 7.47e-03, α = 1.000, ΔJ = 1.066e+00
Iter:  82, Cost:  603.36 →  603.36 ( 2.86e-03), res:  9.38e+00, β= 3.36e-03, α = 0.004, ΔJ = 7.775e+00
Iter:  83, Cost:  603.36 →  600.13 ( 3.23e+00), res:  4.30e+00, β= 1.51e-03, α = 1.000, ΔJ = 3.884e+00
Iter:  84, Cost:  600.13 →  600.08 ( 5.20e-02), res:  4.36e+01, β= 6.81e-04, α = 0.004, ΔJ = 3.309e+01
Iter:  85, Cost:  600.08 →  596.90 ( 3.18e+00), res:  5.38e+01, β= 3.06e-04, α = 0.125, ΔJ = 3.286e+01
Iter:  86, Cost:  596.90 →  596.88 ( 1.85e-02), res:  1.76e+02, β= 1.38e-04, α = 0.004, ΔJ = 1.070e+02
Iter:  87, Cost:  596.88 →  595.84 ( 1.05e+00), res:  1.74e+02, β= 6.21e-05, α = 0.016, ΔJ = 7.675e+01
Iter:  88, Cost:  595.84 →  595.76 ( 7.80e-02), res:  3.26e+02, β= 2.79e-05, α = 0.002, ΔJ = 1.549e+02
Iter:  89, Cost:  595.76 →  642.55 (-4.68e+01), res:  2.54e+02, β= 9.00e-01, α = 0.000, ΔJ = 1.091e+02
Iter:  90, Cost:  642.55 →  612.45 ( 3.01e+01), res:  3.52e+00, β= 4.05e-

Iter: 161, Cost:  593.73 →  593.65 ( 8.42e-02), res:  2.94e+01, β= 3.06e-04, α = 0.008, ΔJ = 1.554e+01
Iter: 162, Cost:  593.65 →  593.50 ( 1.52e-01), res:  6.51e+01, β= 1.38e-04, α = 0.008, ΔJ = 2.076e+01
Iter: 163, Cost:  593.50 →  593.57 (-7.08e-02), res:  1.75e+02, β= 9.00e-01, α = 0.000, ΔJ = 1.141e+02
Iter: 164, Cost:  593.57 →  593.42 ( 1.49e-01), res:  2.23e-01, β= 4.05e-01, α = 1.000, ΔJ = 2.358e-01
Iter: 165, Cost:  593.42 →  593.37 ( 4.93e-02), res:  1.27e-01, β= 1.82e-01, α = 1.000, ΔJ = 6.421e-02
Iter: 166, Cost:  593.37 →  593.31 ( 6.01e-02), res:  2.72e-01, β= 8.20e-02, α = 0.500, ΔJ = 1.184e-01
Iter: 167, Cost:  593.31 →  593.29 ( 1.63e-02), res:  5.81e-01, β= 3.69e-02, α = 0.062, ΔJ = 2.399e-01
Iter: 168, Cost:  593.29 →  593.28 ( 8.36e-03), res:  1.21e+00, β= 1.66e-02, α = 0.016, ΔJ = 4.701e-01
Iter: 169, Cost:  593.28 →  593.28 ( 4.18e-03), res:  2.35e+00, β= 7.47e-03, α = 0.004, ΔJ = 9.128e-01
Iter: 170, Cost:  593.28 →  1062.69 (-4.69e+02), res:  4.19e+00, β= 9.00e

Iter: 241, Cost:  559.47 →  559.24 ( 2.27e-01), res:  1.36e+01, β= 6.81e-04, α = 0.125, ΔJ = 4.882e+00
Iter: 242, Cost:  559.24 →  559.18 ( 5.65e-02), res:  4.21e+01, β= 3.06e-04, α = 0.016, ΔJ = 1.779e+01
Iter: 243, Cost:  559.18 →  558.39 ( 7.96e-01), res:  5.48e+01, β= 1.38e-04, α = 0.062, ΔJ = 1.447e+01
Iter: 244, Cost:  558.39 →  558.08 ( 3.03e-01), res:  2.03e+02, β= 6.21e-05, α = 0.016, ΔJ = 5.038e+01
Iter: 245, Cost:  558.08 →  567.11 (-9.02e+00), res:  1.59e+02, β= 9.00e-01, α = 0.000, ΔJ = 4.219e+01
Iter: 246, Cost:  567.11 →  566.65 ( 4.58e-01), res:  2.01e+00, β= 4.05e-01, α = 0.031, ΔJ = 1.083e+01
Iter: 247, Cost:  566.65 →  566.62 ( 2.75e-02), res:  2.44e+00, β= 1.82e-01, α = 0.002, ΔJ = 1.217e+01
Iter: 248, Cost:  566.62 →  597.76 (-3.11e+01), res:  2.74e+00, β= 1.64e+00, α = 0.000, ΔJ = 1.379e+01
Iter: 249, Cost:  597.76 →  563.30 ( 3.45e+01), res:  2.93e+00, β= 7.38e-01, α = 1.000, ΔJ = 4.247e+01
Iter: 250, Cost:  563.30 →  562.39 ( 9.07e-01), res:  6.62e-01, β= 3.32e-

Iter: 321, Cost:  534.81 →  533.99 ( 8.17e-01), res:  2.70e+02, β= 2.79e-05, α = 0.008, ΔJ = 2.716e+02
Iter: 322, Cost:  533.99 →  532.81 ( 1.18e+00), res:  4.18e+02, β= 1.26e-05, α = 0.031, ΔJ = 3.189e+02
Iter: 323, Cost:  532.81 →  533.22 (-4.10e-01), res:  5.31e+02, β= 9.00e-01, α = 0.000, ΔJ = 4.063e+02
Iter: 324, Cost:  533.22 →  521.42 ( 1.18e+01), res:  1.37e+00, β= 4.05e-01, α = 1.000, ΔJ = 1.905e+01
Iter: 325, Cost:  521.42 →  515.94 ( 5.48e+00), res:  1.50e+00, β= 1.82e-01, α = 1.000, ΔJ = 8.381e+00
Iter: 326, Cost:  515.94 →  512.83 ( 3.11e+00), res:  1.68e+00, β= 8.20e-02, α = 1.000, ΔJ = 7.364e+00
Iter: 327, Cost:  512.83 →  508.40 ( 4.44e+00), res:  3.42e+00, β= 3.69e-02, α = 1.000, ΔJ = 1.379e+01
Iter: 328, Cost:  508.40 →  499.02 ( 9.38e+00), res:  4.71e+00, β= 1.66e-02, α = 1.000, ΔJ = 2.275e+01
Iter: 329, Cost:  499.02 →  498.34 ( 6.76e-01), res:  8.43e+00, β= 7.47e-03, α = 0.125, ΔJ = 3.794e+01
Iter: 330, Cost:  498.34 →  496.05 ( 2.29e+00), res:  1.99e+01, β= 3.36e-

Iter: 401, Cost:  2347.76 →  2325.24 ( 2.25e+01), res:  4.29e+02, β= 6.81e-04, α = 0.008, ΔJ = 3.312e+03
Iter: 402, Cost:  2325.24 →  2052.36 ( 2.73e+02), res:  8.27e+02, β= 3.06e-04, α = 0.250, ΔJ = 3.126e+03
Iter: 403, Cost:  2052.36 →  2064.97 (-1.26e+01), res:  5.25e+02, β= 9.00e-01, α = 0.000, ΔJ = 3.076e+03
Iter: 404, Cost:  2064.97 →  2075.53 (-1.06e+01), res:  9.14e+00, β= 8.10e+00, α = 0.000, ΔJ = 1.387e+03
Iter: 405, Cost:  2075.53 →  2017.90 ( 5.76e+01), res:  3.01e+00, β= 3.65e+00, α = 0.062, ΔJ = 7.460e+02
Iter: 406, Cost:  2017.90 →  1584.56 ( 4.33e+02), res:  4.10e+00, β= 1.64e+00, α = 0.250, ΔJ = 9.154e+02
Iter: 407, Cost:  1584.56 →  1498.20 ( 8.64e+01), res:  3.76e+00, β= 7.38e-01, α = 0.125, ΔJ = 5.916e+02
Iter: 408, Cost:  1498.20 →  1464.73 ( 3.35e+01), res:  5.63e+00, β= 3.32e-01, α = 0.062, ΔJ = 7.318e+02
Iter: 409, Cost:  1464.73 →  1448.84 ( 1.59e+01), res:  1.22e+01, β= 1.49e-01, α = 0.062, ΔJ = 9.376e+02
Iter: 410, Cost:  1448.84 →  1422.89 ( 2.59e+01), res: 

LoadError: PosDefException: matrix is not positive definite; Cholesky factorization failed.

In [37]:
render(vis)

MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), UInt8[0x83, 0xa7, 0x6f, 0x70, 0x74, 0x69, 0x6f, 0x6e, 0x73, 0x82  …  0x5f, 0x61, 0x6e, 0x69, 0x6d, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))))), Set(Any[WebSocket(server, CONNECTED), WebSocket(server, CONNECTED), WebSocket(server, CONNECTED), WebSocket(server, CONNECTED)]), ip"127.0.0.1", 8700))

In [38]:
visualize!(vis, prob.model, prob.tf, Ximm[1:160])

LoadError: UndefVarError: Ximm not defined